Profit for first box:

(Multiplier * 10000)/(I + 100*P)

Profit for second box:

(Multiplier * 10000)/(I + 100*P) - 50,000

(Multiplier- 5I)/500 > P for second box to be profitable

In [97]:
import numpy as np
from scipy.optimize import minimize
import itertools
from tqdm.auto import tqdm
from random import randint

In [ ]:
containers = np.array([(10,1),(80,6),(37,3),(17,1),(90,10),(31,2),(50,4),(20,2),(73,4),(89,8)])

In [4]:
p_values = [(container[0]-5*container[1])/500 for container in containers]

In [5]:
p_values

[np.float64(0.01),
 np.float64(0.1),
 np.float64(0.044),
 np.float64(0.024),
 np.float64(0.08),
 np.float64(0.042),
 np.float64(0.06),
 np.float64(0.02),
 np.float64(0.106),
 np.float64(0.098)]

In [6]:
# First box
worst_case_scenarios = [(10000*container[0])/(container[1]+100) for container in containers]

In [7]:
worst_case_scenarios

[np.float64(990.0990099009902),
 np.float64(7547.169811320755),
 np.float64(3592.233009708738),
 np.float64(1683.1683168316831),
 np.float64(8181.818181818182),
 np.float64(3039.2156862745096),
 np.float64(4807.692307692308),
 np.float64(1960.7843137254902),
 np.float64(7019.2307692307695),
 np.float64(8240.74074074074)]

In [30]:

def minimize_ev(multiplier_1,individuals_1, multiplier_2, individuals_2):
    def function(x):
        p1,p2 = x
        return (multiplier_1 * 10000)/(individuals_1 + 100*p1) + (multiplier_2 * 10000)/(individuals_2 + 100*p2) - 50000

    bounds = [(0,1),(0,1)]
    constraints = [{
    'type': 'ineq',
    'fun': lambda x: 1 - x[0] - x[1]
     }]
    x0 = [1,1]
    return minimize(fun=function,x0=x0,bounds=bounds,constraints=constraints)





In [31]:
result = minimize_ev(10,1,80,6)

In [44]:
container_combos = list(itertools.combinations(containers,2))

In [45]:
len(container_combos)

45

In [46]:
for combo in container_combos:
    if combo[0][0] == combo[1][0] and combo[0][1] == combo[1][1]:
        print(combo)

In [47]:
container_combos = [[combo[0][0],combo[0][1],combo[1][0],combo[1][1]] for combo in container_combos]

In [48]:
len(container_combos)

45

In [49]:
minimums = []
for combo in tqdm(container_combos,"Progress"):
    result = minimize_ev(combo[0],combo[1],combo[2],combo[3])
    minimums.append((result.fun, result.x[0], result.x[1], combo[0], combo[1], combo[2], combo[3]))


Progress: 100%|██████████| 45/45 [00:00<00:00, 218.12it/s]


In [59]:
[",".join([str(i) for i in e[3:]]) for e in sorted(minimums)[::-1][:10]]

['90,10,89,8',
 '80,6,89,8',
 '80,6,90,10',
 '73,4,89,8',
 '90,10,73,4',
 '80,6,73,4',
 '50,4,89,8',
 '90,10,50,4',
 '80,6,50,4',
 '50,4,73,4']

In [75]:
ratios = [container[0]/container[1] for container in containers]

In [76]:
ratios = np.array([ratio/sum(ratios) for ratio in ratios])

In [77]:
ratios

array([0.07749435, 0.1033258 , 0.09557636, 0.13174039, 0.06974491,
       0.12011624, 0.09686794, 0.07749435, 0.14142719, 0.08621246])

In [78]:
sum(ratios)

np.float64(0.9999999999999999)

In [66]:
container_combos[:5]

[[np.int64(10), np.int64(1), np.int64(80), np.int64(6)],
 [np.int64(10), np.int64(1), np.int64(37), np.int64(3)],
 [np.int64(10), np.int64(1), np.int64(17), np.int64(1)],
 [np.int64(10), np.int64(1), np.int64(90), np.int64(10)],
 [np.int64(10), np.int64(1), np.int64(31), np.int64(2)]]

In [71]:
indexed_combos = list(itertools.combinations([*range(10)],2))

In [72]:
len(indexed_combos)

45

In [83]:
indexed_combos+=[(j,i) for i,j in indexed_combos]

In [84]:
len(indexed_combos)

90

In [109]:
def evaluate_combo(containers, combo,ratios):
    multiplier_1 = containers[combo[0]][0]
    individuals_1 = containers[combo[0]][1]
    multiplier_2 = containers[combo[1]][0]
    individuals_2 = containers[combo[1]][1]
    p1 = ratios[combo[0]]
    p2 = ratios[combo[1]]
    return (10000*multiplier_1)/(individuals_1 + 100*p1) + (10000*multiplier_2)/(individuals_2 + 100*p2) - 50000
def maximize_with_prior(containers,ratios,power,combos,top):
   raised_prior = ratios**power
   values = []
   for combo in combos:
         value = evaluate_combo(containers,combo,raised_prior)
         values.append((float(value),containers[combo[0]],containers[combo[1]],combo))
#    print(values)
   return sorted(values,key=lambda x: x[0],reverse=True)[:top]

In [111]:
maximize_with_prior(containers,ratios,1,indexed_combos,10)

[(56566.65857332667, array([90, 10]), array([89,  8]), (4, 9)),
 (56566.65857332667, array([89,  8]), array([90, 10]), (9, 4)),
 (52527.77556567686, array([80,  6]), array([89,  8]), (1, 9)),
 (52527.77556567686, array([89,  8]), array([80,  6]), (9, 1)),
 (52002.58551051645, array([80,  6]), array([90, 10]), (1, 4)),
 (52002.58551051645, array([90, 10]), array([80,  6]), (4, 1)),
 (43782.451686636225, array([73,  4]), array([89,  8]), (8, 9)),
 (43782.451686636225, array([89,  8]), array([73,  4]), (9, 8)),
 (43257.26163147582, array([90, 10]), array([73,  4]), (4, 8)),
 (43257.26163147582, array([73,  4]), array([90, 10]), (8, 4))]

In [112]:
maximize_with_prior(containers,ratios,2,indexed_combos,10)

[(184855.565779821, array([80,  6]), array([73,  4]), (1, 8)),
 (184855.565779821, array([73,  4]), array([80,  6]), (8, 1)),
 (173456.0302613695, array([73,  4]), array([89,  8]), (8, 9)),
 (173456.0302613695, array([89,  8]), array([73,  4]), (9, 8)),
 (172911.92568018922, array([50,  4]), array([73,  4]), (6, 8)),
 (172911.92568018922, array([73,  4]), array([50,  4]), (8, 6)),
 (166208.23067840416, array([37,  3]), array([73,  4]), (2, 8)),
 (166208.23067840416, array([73,  4]), array([37,  3]), (8, 2)),
 (164984.95172770994, array([80,  6]), array([89,  8]), (1, 9)),
 (164984.95172770994, array([89,  8]), array([80,  6]), (9, 1))]

In [116]:
ratios**1

array([0.07749435, 0.1033258 , 0.09557636, 0.13174039, 0.06974491,
       0.12011624, 0.09686794, 0.07749435, 0.14142719, 0.08621246])

In [113]:
[maximize_with_prior(containers,ratios,i,indexed_combos,10)[0] for i in range(1,11)]

[(56566.65857332667, array([90, 10]), array([89,  8]), (4, 9)),
 (184855.565779821, array([80,  6]), array([73,  4]), (1, 8)),
 (263086.19705841073, array([31,  2]), array([73,  4]), (5, 8)),
 (295721.85937141906, array([17,  1]), array([73,  4]), (3, 8)),
 (301570.2878827861, array([17,  1]), array([73,  4]), (3, 8)),
 (302374.67317420046, array([17,  1]), array([73,  4]), (3, 8)),
 (302483.12961209484, array([17,  1]), array([73,  4]), (3, 8)),
 (302497.72736340784, array([17,  1]), array([73,  4]), (3, 8)),
 (302499.69352634053, array([17,  1]), array([73,  4]), (3, 8)),
 (302499.95862459263, array([17,  1]), array([73,  4]), (3, 8))]